In [1]:
from pathlib import Path

In [2]:
path = Path("piano_music/")

In [3]:
from data import AudioMP3Dataset, collate_fn

In [4]:
dataset = AudioMP3Dataset(path)

In [5]:
from torch.utils.data import DataLoader

In [6]:
dl = DataLoader(dataset, batch_size=1, collate_fn=collate_fn)

In [7]:
from model import WaveNet

In [8]:
res_layers = 4
res_blocks = 1
casual_layers = 1
kernal_size = 3

model = WaveNet(res_layers, res_blocks, casual_layers, kernal_size=kernal_size)

In [9]:
def calculateReceptiveField(res_layers, res_blocks, kernal_size):
    dialtions = [2**i for i in range(res_layers)] * res_blocks
    return (kernal_size - 1) * sum(dialtions) + 1 

In [10]:
receptive_field = calculateReceptiveField(res_layers, res_blocks, 3)
receptive_field

31

In [11]:
for inputs in dl: break

In [12]:
inputs.shape

torch.Size([88, 1, 50000])

In [13]:
from torch.optim import SGD
from torch.nn import BCELoss

optimizer = SGD(model.parameters(), lr=0.01)
loss_fn = BCELoss()

In [14]:
def one_mask_step(model, inputs, i, optimizer, loss_fn, quantize_channels):
    pass

In [15]:
import torch
from preprocess import muLaw
from torch.nn.functional import one_hot

In [16]:
masked_inputs = inputs[:,:,:receptive_field+1]

optimizer.zero_grad()

pred = model(masked_inputs)

targets = muLaw(inputs[:,:,receptive_field+2])
one_hot_target = one_hot(targets, num_classes=quantize_channels)
one_hot_target = one_hot_target.transpose(1, 2).to(torch.float)

loss = loss_fn(pred, one_hot_target)
loss.backward()

optimizer.step()
print(f"Loss: {loss.item()}")

NameError: name 'quantize_channels' is not defined

In [ ]:
import torch
from preprocess import muLaw
from torch.nn.functional import one_hot

def one_mask_step(model, inputs, mask_index, optimizer, 
                  loss_fn, quantize_channels, verbose=True):
    masked_inputs = inputs[:,:,:mask_index]

    optimizer.zero_grad()

    pred = model(masked_inputs)

    targets = muLaw(inputs[:,:,mask_index+1])
    one_hot_target = one_hot(targets, num_classes=quantize_channels)
    one_hot_target = one_hot_target.transpose(1, 2).to(torch.float)

    loss = loss_fn(pred, one_hot_target)
    loss.backward()

    optimizer.step()
    if verbose:
        print(f"Loss: {loss.item()}")

In [ ]:
one_mask_step(model, inputs, receptive_field+1, optimizer, loss_fn, 256)

In [ ]:
for i in range(receptive_field+1, inputs.size(-1)):
    one_mask_step(model, inputs, i, optimizer, loss_fn, 256)
    break